<img src="img/RAG.png"  style="float: left; margin-right: 10px;" height=1500/>

In [1]:
! pip install transformers 
! pip install chromadb 
! pip install wikipedia 
! pip install langchain 
! pip install oci 
! pip install langchain_community

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# Import the langchain libraries 

In [2]:
from langchain_community.document_loaders import WikipediaLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from llms.oci_model_wrapper import OCIModelWrapper

#### Get the OCI LLM and Embedding model 

In [3]:
oci_wrapper = OCIModelWrapper()
llm = oci_wrapper.llm
embedding = oci_wrapper.embeddings


### Document Loader
###### [Documentation Link](https://python.langchain.com/docs/modules/data_connection/document_loaders/)


In [4]:
docs = WikipediaLoader(query = "Oracle Corporation").load()
print(docs[0].metadata)  # meta-information of the Document
docs[0].page_content[:300]  # a content of the Document


{'title': 'Oracle Corporation', 'summary': "Oracle Corporation is an American multinational computer technology company headquartered in Austin, Texas, United States. In 2020, Oracle was the third-largest software company in the world by revenue and market capitalization. In 2023, the company’s seat in Forbes Global 2000 was 80. The company sells database software (particularly the Oracle Database) and cloud computing. Oracle's core application software is a suite of enterprise software products, such as enterprise resource planning (ERP) software, human capital management (HCM) software, customer relationship management (CRM) software, enterprise performance management (EPM) software, Customer Experience Commerce (CX Commerce) and supply chain management (SCM) software.", 'source': 'https://en.wikipedia.org/wiki/Oracle_Corporation'}


'Oracle Corporation is an American multinational computer technology company headquartered in Austin, Texas, United States. In 2020, Oracle was the third-largest software company in the world by revenue and market capitalization. In 2023, the company’s seat in Forbes Global 2000 was 80. The company s'

##### Document Transformation
##### To accommodate LLMs' token and input size limits, this approach chunks large documents, ensuring they can be summarized without exceeding LLM constraints.
###### [Link](https://python.langchain.com/docs/modules/data_connection/document_transformers/)

In [5]:
splitted_docs = RecursiveCharacterTextSplitter(chunk_size = 900, chunk_overlap = 20, length_function=len)

chunks = splitted_docs.split_documents(docs)

In [6]:
print(f"Total Chunks created {len(chunks)}")
for i, _ in enumerate(chunks):
    print(f"chunk# {i}, size: {chunks[i]}")

Total Chunks created 159
chunk# 0, size: page_content="Oracle Corporation is an American multinational computer technology company headquartered in Austin, Texas, United States. In 2020, Oracle was the third-largest software company in the world by revenue and market capitalization. In 2023, the company’s seat in Forbes Global 2000 was 80. The company sells database software (particularly the Oracle Database) and cloud computing. Oracle's core application software is a suite of enterprise software products, such as enterprise resource planning (ERP) software, human capital management (HCM) software, customer relationship management (CRM) software, enterprise performance management (EPM) software, Customer Experience Commerce (CX Commerce) and supply chain management (SCM) software.\n\n\n== History ==" metadata={'title': 'Oracle Corporation', 'summary': "Oracle Corporation is an American multinational computer technology company headquartered in Austin, Texas, United States. In 2020, Or

### Store the embedding data in a vector store (Chroma DB)
###### [Link](https://python.langchain.com/docs/modules/data_connection/text_embedding/)

In [7]:
persist_directory = 'demo_db_1'

In [13]:
# Current limitation of the 96 elements in input array
chunk_size = 96

# Calculate the total number of chunks needed to process all elements
# This is simply the length of the chunks array divided by the chunk size
num_chunks = len(chunks) // chunk_size

# If there are any remaining elements after forming full chunks, add one more chunk for them
if len(chunks) % chunk_size > 0:
    num_chunks += 1

for i in range(num_chunks):
    # Calculate the start index for the current chunk
    start_idx = i * chunk_size
    
    # Calculate the end index for the current chunk
    # This is the start index plus the chunk size, but it should not exceed the length of the chunks array
    end_idx = min(start_idx + chunk_size, len(chunks))
    
    # Slice the chunks array to get the current chunk
    current_chunk = chunks[start_idx:end_idx]
    
    # Process the current chunk
    vectordb = Chroma.from_documents(documents=current_chunk, embedding=embedding,persist_directory=persist_directory)
    vectordb.persist()
    vectordb = None



#### Load the ChromaDB from a local file

In [14]:
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

##### Get the DB to check for a valid file

In [15]:
vectordb.get()

{'ids': ['012c7cd0-c91e-47cb-95c7-c672b3acf2bd',
  '01af4b91-be20-4025-bb2e-c839b488a5cb',
  '01b67486-2790-45de-b3fa-7c846b2a111d',
  '025c3640-1de1-43f7-bb6e-53a519aafd7e',
  '04883255-543d-465c-b2fd-be3286340576',
  '04a0f889-968e-4489-bdf0-32315f28062b',
  '0528be2b-0e3f-48f9-a527-33eb4aec7c1e',
  '0532a5e1-1e00-4f12-a637-e234c9c73e52',
  '055e3108-0661-4970-966b-5726db7e741c',
  '07226ae3-aeae-4cd9-b259-d7a56b01af46',
  '07e136bf-3c98-4c4b-b3b3-2193f19a8970',
  '07fa44ce-9a35-485e-82a2-9fb4a77c9146',
  '098b24cc-b2bb-4f98-9458-28849d16ce50',
  '0a9d72af-336d-4494-9df5-7221042712c1',
  '0b4e38b9-df6e-4f83-b288-a83f6e28a6a4',
  '0b72e02b-d079-4805-bfcc-5011574d7cf3',
  '0ca3ae76-e910-41ed-84c5-fbf73c93b7ba',
  '0ca91317-7c09-4570-adae-46137360717e',
  '0d4dcbc6-d01a-452a-a3ce-ebb8b32b5c95',
  '0f7568eb-5ef4-4d3f-95c8-cd70bd5b6bb8',
  '10b919a0-b1b2-4174-9512-98b452a3e622',
  '11b585bd-72ea-4781-a4fd-92aa3b3ce9aa',
  '125ac7d4-8d48-473f-8383-65549a48758d',
  '125f1036-eb56-4492-bf00-

## Retrievers
### Use retriever to return relevant document
###### [Link](https://python.langchain.com/docs/modules/data_connection/retrievers/)

In [16]:
retriever = vectordb.as_retriever()
query = "Who was the first CEO of Oracle?"
docs = vectordb.similarity_search(query)
print(docs)

[Document(page_content='Larry Ellison co-founded Oracle Corporation in 1977 with Bob Miner and Ed Oates under the name Software Development Laboratories (SDL). Ellison took inspiration from the 1970 paper written by Edgar F. Codd on relational database management systems (RDBMS) named "A Relational Model of Data for Large Shared Data Banks." He heard about the IBM System R database from an article in the IBM Research Journal provided by Oates. Ellison wanted to make Oracle\'s product compatible with System R, but failed to do so as IBM kept the error codes for their DBMS a secret. SDL changed its name to Relational Software, Inc (RSI) in 1979, then again to Oracle Systems Corporation in 1983, to align itself more closely with its flagship product Oracle Database. The name also drew from the 1977 CIA project codename, which was also Oracle\'s first customer. At this stage, Bob Miner served as the company\'s senior', metadata={'source': 'https://en.wikipedia.org/wiki/Oracle_Corporation',

### Run test querries to return documents

In [17]:
docs = retriever.get_relevant_documents("Who was the first CEO of Oracle?")
len(docs)

4

### Pass search [arguments](https://python.langchain.com/docs/modules/data_connection/retrievers/vectorstore)

In [18]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
retriever.search_type

'similarity'

### Create a chain 
###### [Link](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html)

#### [Optional] Create a handler to get verbose information. Helpful while troubleshooting

In [19]:
from langchain.globals import set_verbose, set_debug
set_debug(True)
set_verbose(True)
from langchain.callbacks import StdOutCallbackHandler
handler = StdOutCallbackHandler()

#### RetrievalQA Chain with map_reduce as chain type. Enable callback variable to get verbose output

In [20]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  chain_type= "map_reduce", 
                                  retriever=retriever, 
                                  return_source_documents=True,
                                    #callbacks=[handler],
                                    )

#### Function to return relevant output

In [21]:
def print_output(response):
    # Check if 'result' key exists in the response and print its value
    if 'result' in response:
        print(f"Result: {response['result']} \n\n")
    else:
        print("Result: No result found.\n\n")
    
    # Check if 'source_documents' key exists and it is a list
    if 'source_documents' in response and isinstance(response['source_documents'], list):
        # Iterate through each source document in the list
        for i, src in enumerate(response['source_documents'], start=1):
            # Access 'metadata' directly assuming 'src' is an object with a 'metadata' attribute
            # Check if 'metadata' exists and is a dictionary, then access 'source'
            if hasattr(src, 'metadata') and isinstance(src.metadata, dict):
                source_url = src.metadata.get('source', 'No source found')
            else:
                source_url = 'No source found'
            print(f"Source {i}: {source_url}")
    else:
        print("Source Documents: No source documents found.")
    
    return None

### Query

In [22]:
query = "When did Oracle partner with microsoft? Answer in one line"
llm_response = qa_chain.invoke(query)
print_output(llm_response)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "When did Oracle partner with microsoft? Answer in one line"
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "== External links ==\nOracle SQL Developer Homepage",
      "question": "When did Oracle partner with microsoft? Answer in one line"
    },
    {
      "context": "== External links ==\nOracle SQL Developer Homepage",
      "question": "When did Oracle partner with microsoft? Answer in one line"
    }
  ]
}
[llm/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] Entering LLM run with input:
{
  "prompts": [
    "Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verba

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


C:\ProgramData\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\talemos\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Token indices sequence length is longer than the specified maximum sequence length for this model (1625 > 1024). Running this sequence through the model will result in indexing errors


[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "When did Oracle partner with microsoft? Answer in one line",
  "summaries": " Oracle has partnered with Microsoft on many occasions throughout the years, but one significant collaboration began in 2018 when Oracle announced its move to the Microsoft Azure cloud platform. This partnership was aimed at helping businesses migrate their applications and workloads to the cloud and was seen as a strategic alliance between the two companies. \n\nIs there anything specific you would like to know regarding this partnership or would you like to inquire about another partnership between Oracle and Microsoft? \n\n Oracle has partnered with Microsoft on many occasions throughout the years, but one significant collaboration began in 2018 when Oracle announced its move to the Microsoft Azure cloud platform. This partnership was aimed at helping businesses migrate their ap

[llm/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] [2.68s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Oracle's partnership with Microsoft began when it announced its move to Microsoft Azure cloud platform for businesses in 2018. \n\nWould you like to know more about this partnership or Oracle's other partnerships with Microsoft? ",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] [2.68s] Exiting Chain run with output:
{
  "text": " Oracle's partnership with Microsoft began when it announced its move to Microsoft Azure cloud platform for businesses in 2018. \n\nWould you like to know more about this partnership or Oracle's other partnerships with Microsoft? "
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain] [19.81s] Exiting Chain run with ou

In [23]:
query = "Who is the current CEO of Oracle?"
llm_response = qa_chain.invoke(query)
print_output(llm_response)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Who is the current CEO of Oracle?"
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "In the cloud, Oracle Database competes against the database ser",
      "question": "Who is the current CEO of Oracle?"
    },
    {
      "context": "In the cloud, Oracle Database competes against the database ser",
      "question": "Who is the current CEO of Oracle?"
    }
  ]
}
[llm/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] Entering LLM run with input:
{
  "prompts": [
    "Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\nIn the cloud, Oracle Database competes agains

[llm/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] [2.74s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Unfortunately, I can't provide an answer to this question as there is no relevant information in the given text extract. \n\nIs there anything else you'd like to know?  I can search through other parts of the document for additional information if you'd like. ",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] [2.74s] Exiting Chain run with output:
{
  "text": " Unfortunately, I can't provide an answer to this question as there is no relevant information in the given text extract. \n\nIs there anything else you'd like to know?  I can search through other parts of the document for additional information if you'd like. "
}
[chain/end] [chain:RetrievalQA > cha

In [24]:
query = "What was the original name of Oracle?"
llm_response = qa_chain.invoke(query)
print_output(llm_response)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What was the original name of Oracle?"
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "In 1979, the company renamed itself Relational Software Inc. Ellison had heard about the IBM System R database, also based on Codd's theories, and wanted Oracle to achieve compatibility with it, but IBM made this impossible by refusing to share System R's error codes. The initial release of the Oracle Database in 1979 was called Oracle version 2; there was no Oracle version 1. In 1983, the company officially became Oracle Systems Corporation after its flagship product. In 1990, Oracle laid off 10% of its workforce (about 400 people) because it was losing money. This crisis, which almost resulted in the compan

[llm/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] [1.60s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " The original name of Oracle was Relational Software Inc. ",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] [1.60s] Exiting Chain run with output:
{
  "text": " The original name of Oracle was Relational Software Inc. "
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain] [8.58s] Exiting Chain run with output:
{
  "output_text": " The original name of Oracle was Relational Software Inc. "
}
[chain/end] [chain:RetrievalQA] [9.15s] Exiting Chain run with output:
[outputs]
Result:  The original name of Oracle was Relational Software Inc.  


Source 1: https://en.wikipedia.org/wiki/Larry_Ellison
Source 2: https://en.wikipedia.org/wiki/Larry_Elliso

In [25]:
query = "What products does Oracle sell? Anser the question in bulltet points."
llm_response = qa_chain.invoke(query)
print_output(llm_response)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What products does Oracle sell? Anser the question in bulltet points."
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "== External links ==\nOracle SQL Developer Homepage",
      "question": "What products does Oracle sell? Anser the question in bulltet points."
    },
    {
      "context": "== External links ==\nOracle SQL Developer Homepage",
      "question": "What products does Oracle sell? Anser the question in bulltet points."
    }
  ]
}
[llm/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] Entering LLM run with input:
{
  "prompts": [
    "Use the following portion of a long document to see if any of the text is relevant to answer the question.

[llm/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] [9.17s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Oracle sells a variety of products and services that are divided into four main divisions:\n\n1. Cloud Infrastructure\n2. Cloud Platform\n3. Software\n4. Hardware\n\nOracle's Cloud Infrastructure offers cloud computing services such as computing, storage and networking. It provides services including Autonomous Database, Virtual Machines and Storage Services. \n\nOracle Cloud Platform offers a range of services that allow businesses to develop and deploy cloud-based applications. Services include Application Development, Business Analytics, Content and Experience, Integration, Management and Security. \n\nOracle sells a range of software solutions including ERP, HCM, CRM, SCM and many others. Some of their key products include Oracle Database, Java, ERP Cloud, Fusion Middleware and PeopleSoft. \n\nOracle 

In [26]:
query = "When did Oracle come into an existence? Answer in one line"
llm_response = qa_chain.invoke(query)
print_output(llm_response)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "When did Oracle come into an existence? Answer in one line"
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "== Early career and Oracle ==",
      "question": "When did Oracle come into an existence? Answer in one line"
    },
    {
      "context": "== Early career and Oracle ==",
      "question": "When did Oracle come into an existence? Answer in one line"
    }
  ]
}
[llm/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] Entering LLM run with input:
{
  "prompts": [
    "Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n== Early career and Oracle ==\nQuestio

[llm/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] [3.84s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Oracle was founded in 1982. \n\nWould you like to know more about the company?  I can provide a summary of its history and key developments if you'd like. ",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] [3.85s] Exiting Chain run with output:
{
  "text": " Oracle was founded in 1982. \n\nWould you like to know more about the company?  I can provide a summary of its history and key developments if you'd like. "
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain] [9.02s] Exiting Chain run with output:
{
  "output_text": " Oracle was founded in 1982. \n\nWould you like to know more about the company?  I can provide a summary of its history and

In [27]:
query = "When did Oracle aquire Cerner?Answer in one line"
llm_response = qa_chain.invoke(query)
print_output(llm_response)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "When did Oracle aquire Cerner?Answer in one line"
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "== External links ==\nOracle SQL Developer Homepage",
      "question": "When did Oracle aquire Cerner?Answer in one line"
    },
    {
      "context": "== External links ==\nOracle SQL Developer Homepage",
      "question": "When did Oracle aquire Cerner?Answer in one line"
    }
  ]
}
[llm/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] Entering LLM run with input:
{
  "prompts": [
    "Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n== External links ==\nOr

[llm/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] [2.68s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Oracle Corporation completed their acquisition of Cerner in June 2022 for an undisclosed sum. \n\nWould you like to know more about this acquisition? \n\nI can tell you about other major acquisitions if you like. ",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] [2.68s] Exiting Chain run with output:
{
  "text": " Oracle Corporation completed their acquisition of Cerner in June 2022 for an undisclosed sum. \n\nWould you like to know more about this acquisition? \n\nI can tell you about other major acquisitions if you like. "
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain] [6.27s] Exiting Chain run with output:
{
  "output_text": " Oracle

In [28]:
query = "Name few hardware products. The answer should be in bullet points"
llm_response = qa_chain.invoke(query)
print_output(llm_response)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Name few hardware products. The answer should be in bullet points"
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "== External links ==\nOracle SQL Developer Homepage",
      "question": "Name few hardware products. The answer should be in bullet points"
    },
    {
      "context": "== External links ==\nOracle SQL Developer Homepage",
      "question": "Name few hardware products. The answer should be in bullet points"
    }
  ]
}
[llm/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] Entering LLM run with input:
{
  "prompts": [
    "Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn an

[llm/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] [4.59s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Sorry, the given text is purely about software and doesn't mention any hardware products. \n\nIf you'd like, I can instead list some examples of hardware products for you. Would you like that? \n\nSome examples of hardware products include:\n\n- Computers\n- Tablets\n- Smartphones\n- Servers\n- Networking devices (switches, routers, modems, etc.)\n- Printers\n- External hard drives\n- CPUs\n- Motherboards\n\nWould you like me to continue? ",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] [4.59s] Exiting Chain run with output:
{
  "text": " Sorry, the given text is purely about software and doesn't mention any hardware products. \n\nIf you'd like, I can in

In [29]:
query = '''
Could you analyze and discuss the ethical framework and values that guide Oracle Corporation? Specifically, 
examine how these principles influence Oracle's decision-making processes, 
corporate policies, and its approach to social responsibility. 
Provide examples to illustrate where the company's 'moral compass' points, 
especially in situations involving significant ethical dilemmas or decisions.
'''
llm_response = qa_chain.invoke(query)
print_output(llm_response)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "\nCould you analyze and discuss the ethical framework and values that guide Oracle Corporation? Specifically, \nexamine how these principles influence Oracle's decision-making processes, \ncorporate policies, and its approach to social responsibility. \nProvide examples to illustrate where the company's 'moral compass' points, \nespecially in situations involving significant ethical dilemmas or decisions.\n"
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "This is a listing of Oracle Corporation's corporate acquisitions, including acquisitions of both companies and individual products.\nOracle's version does not include value of the acquisition.\nSee also Category:Sun Microsystems acquisitions (

[llm/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] [2.56s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Unfortunately, the information provided is not sufficient to discuss the ethical framework or values that guide Oracle Corporation. \n\nWould you like to provide additional details or information regarding Oracle Corporation? ",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] [2.56s] Exiting Chain run with output:
{
  "text": " Unfortunately, the information provided is not sufficient to discuss the ethical framework or values that guide Oracle Corporation. \n\nWould you like to provide additional details or information regarding Oracle Corporation? "
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain] [12.19s] Exiting Chain run with output:


In [30]:
query = '''
Please calculate the total amount Oracle has spent on acquisitions where the purchase price is publicly disclosed. 
Exclude any acquisitions where the purchase price has not been shared. 
Provide the final sum in USD, and break down the calculation using a mathematical equation. 
Ensure the explanation is clear, incorporating each acquisition's cost into the equation to arrive at the total expenditure.
'''
llm_response = qa_chain.invoke(query)
print_output(llm_response)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "\nPlease calculate the total amount Oracle has spent on acquisitions where the purchase price is publicly disclosed. \nExclude any acquisitions where the purchase price has not been shared. \nProvide the final sum in USD, and break down the calculation using a mathematical equation. \nEnsure the explanation is clear, incorporating each acquisition's cost into the equation to arrive at the total expenditure.\n"
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "This is a listing of Oracle Corporation's corporate acquisitions, including acquisitions of both companies and individual products.\nOracle's version does not include value of the acquisition.\nSee also Category:Sun Microsystems acquisitions

[llm/end] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OCIGenAI] [5.00s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Unfortunately, the information provided does not include the purchase prices of Oracle's acquisitions. Without this crucial data, it is impossible to calculate the total amount Oracle has spent on acquisitions where the purchase price is publicly disclosed. \n\nIf you can provide the specific details regarding each acquisition's cost, I would be happy to assist you in determining the overall spending using a clear mathematical equation. \n\nWould you like to provide the necessary information for the calculation? Alternatively, if there is another question I can help you with regarding the document you've shared, please let me know. ",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RetrievalQA > chain:MapReduceDocume